In [1]:
using LinearAlgebra, Random

In [34]:
Random.seed!(1)
X = [sqrt.(diagm(1:3)) + 0.01*randn(3, 3) zeros(3,1);
     zeros(1,4)]
Sigma = X' * X

4×4 Array{Float64,2}:
 1.006        0.00525246   0.0125494  0.0
 0.00525246   1.97635     -0.0264685  0.0
 0.0125494   -0.0264685    3.01943    0.0
 0.0          0.0          0.0        0.0

In [48]:
n = 4

4

In [37]:
X_update = [1; 0; 0; 1] / sqrt(2)
Sigma_update = [
    zeros(3,3) (X[:, 1:3]' * X_update)
    (X[:, 1:3]' * X_update)' norm(X_update)^2
]

4×4 Array{Float64,2}:
 0.0        0.0        0.0         0.709209
 0.0        0.0        0.0        -7.3859e-5
 0.0        0.0        0.0         0.0162287
 0.709209  -7.3859e-5  0.0162287   1.0

In [16]:
eigen_Sigma = eigen(Hermitian(Sigma))
V_Sigma = eigen_Sigma.vectors[:, [2; 3; 4; 1]]

4×4 Array{Float64,2}:
  0.999965    -0.00574192   0.00616242  0.0
 -0.00558417  -0.999664    -0.0253176   0.0
 -0.00630572  -0.0252823    0.99966     0.0
  0.0          0.0          0.0         1.0

In [41]:
R = V_Sigma' * (Sigma + Sigma_update) * V_Sigma

4×4 Array{Float64,2}:
  1.0059       -2.40778e-16   6.02514e-18   0.709082
 -2.41097e-16   1.97571      -1.1152e-17   -0.00440869
  7.34811e-18  -1.26622e-18   3.02018       0.0205955
  0.709082     -0.00440869    0.0205955     1.0

## Formula for Cholesky decomposition of R

In [47]:
cholesky(Hermitian(R)).U'

4×4 Adjoint{Float64,UpperTriangular{Float64,Array{Float64,2}}}:
  1.00294       0.0          0.0        0.0
 -2.40071e-16   1.4056       0.0        0.0
  6.00745e-18  -7.93397e-18  1.73787    0.0
  0.707001     -0.00313652   0.0118511  0.707107

In [62]:
# R_ii = L_ii^2, i < n
# R_nj = L_nj * L_jj, j < n
# R_nn = sum L_nj^2 (sqrt: sign of L_nn does not matter)
L = diagm(sqrt.(diag(R[1:n-1, 1:n-1])))
L = [L; R[n, 1:n-1]' ./ diag(L)']
L = [L [zeros(n-1); sqrt(R[n,n] - norm(L[n,1:n-1])^2)]]

4×4 Array{Float64,2}:
 1.00294    0.0         0.0        0.0
 0.0        1.4056      0.0        0.0
 0.0        0.0         1.73787    0.0
 0.707001  -0.00313652  0.0118511  0.707107

In [63]:
eigen(Hermitian(L * L'))

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
4-element Array{Float64,1}:
 0.2937763393794084
 1.7118381297803817
 1.975752045145706
 3.020414589471855
vectors:
4×4 Array{Float64,2}:
 -0.705582     0.708581    0.00704133    0.00409349
  0.0018574    0.0117863  -0.999929     -4.90776e-5
 -0.00535289  -0.0111049  -0.000189916   0.999924
  0.708605     0.705443    0.00963087    0.0116297

In [64]:
eigen(Hermitian(L' * L))

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
4-element Array{Float64,1}:
 0.2937763393794083
 1.7118381297803835
 1.9757520451457076
 3.0204145894718555
vectors:
4×4 Array{Float64,2}:
 -0.381311     -0.924367     0.00986835    0.00709333
  0.000716229  -0.0109711   -0.99994      -6.06814e-5
 -0.00166948    0.00836047  -0.000153607   0.999964
  0.924445     -0.381255     0.0048449     0.00473173

## Formula for similarity transformation from LL' to L'L

Unitary matrix W: The first column of W must be orthogonal to columns 2 through n-1 of L, and so on.

In [72]:
L

4×4 Array{Float64,2}:
 1.00294    0.0         0.0        0.0
 0.0        1.4056      0.0        0.0
 0.0        0.0         1.73787    0.0
 0.707001  -0.00313652  0.0118511  0.707107

In [74]:
rot = Matrix(1.0I, n, n)
LT = L
theta = atan(-LT[4,1], LT[1,1])
rot_i = Matrix(1.0I, n, n)
rot_i[1,1] = cos(theta)
rot_i[1,4] = -sin(theta)
rot_i[4,1] = sin(theta)
rot_i[4,4] = cos(theta)
rot_i * L

4×4 Array{Float64,2}:
  1.22709      -0.00180714  0.00682811  0.407407
  0.0           1.4056      0.0         0.0
  0.0           0.0         1.73787     0.0
 -2.23816e-17  -0.00256359  0.00968629  0.577944

In [73]:
rot_i

4×4 Array{Float64,2}:
  0.70716   0.0  0.0  0.707054
  0.0       1.0  0.0  0.0
  0.0       0.0  1.0  0.0
 -0.707054  0.0  0.0  0.70716